<a href="https://colab.research.google.com/github/bachsykhang/StockPredictVinamilk/blob/main/Vinamilk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bước 1: Import thư viện

In [ ]:
import pandas as pd #đọc dữ liệu
import numpy as np #xử lý dữ liệu
import matplotlib.pyplot as plt #vẽ biểu đồ
from sklearn.preprocessing import MinMaxScaler #chuẩn hóa dữ liệu
from keras.callbacks import ModelCheckpoint #lưu lại huấn luyện tốt nhất
from tensorflow.keras.models import load_model #tải mô hình

#các lớp để xây dựng mô hình
from keras.models import Sequential #đầu vào
from keras.layers import LSTM #học phụ thuộc
from keras.layers import Dropout #tránh học tủ
from keras.layers import Dense #đầu ra

#kiểm tra độ chính xác của mô hình
from sklearn.metrics import r2_score #đo mức độ phù hợp
from sklearn.metrics import mean_absolute_error #đo sai số tuyệt đối trung bình
from sklearn.metrics import mean_absolute_percentage_error #đo % sai số tuyệt đối trung bình

# Bước 2: Đọc dữ liệu

In [ ]:
#tạo quyền truy cập với google drive
from google.colab import drive
drive.mount('/drive')

# Xóa hai dòng "KL" và "Thay đổi %" từ DataFrame dataSet
# df = df.drop(columns=["Vol.", "Change %"])


In [ ]:
# đọc dữ liệu từ file csv
df = pd.read_csv('/drive/My Drive/DoAnDeepLearning/Amazon.csv')

# Hiển thị lại DataFrame sau khi xóa
print(df)

# Bước 3: Mô tả dữ liệu

In [ ]:
#định dạng cấu trúc thời gian
df["Date"] = pd.to_datetime(df.Date,format="%m/%d/%Y")

In [ ]:
#kích thước dữ liệu
df.shape

In [ ]:
#dữ liệu 5 dòng đầu
df.head()

In [ ]:
#xác định kiểu dữ liệu
df.info()

In [ ]:
#mô tả bộ dữ liệu
df.describe()

# Bước 4: Tiền xử lý dữ liệu

In [ ]:
from matplotlib.dates import YearLocator, DateFormatter, MonthLocator # Thêm MonthLocator vào để sửa lỗi

# Chuyển đổi cột "Date" sang dạng datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Sắp xếp lại dữ liệu theo thứ tự thời gian
df = df.sort_values(by='Date')

# Chuyển đổi định dạng các cột giá thành số thực
df['Price'] = df['Price'].astype(float)
df['Open'] = df['Open'].astype(float)
df['High'] = df['High'].astype(float)
df['Low'] = df['Low'].astype(float)

# Lấy thông tin năm từ cột "Date"
df['Năm'] = df['Date'].dt.year




In [ ]:
# Tạo đồ thị giá Price qua các năm
plt.figure(figsize=(10, 5))
plt.plot(df['Date'], df['Price'], label='Giá Close', color='red')
plt.xlabel('Năm')
plt.ylabel('Giá Close')
plt.title('Biểu đồ giá Price của AMAZON qua các năm')
plt.legend(loc='best')

# Định dạng đồ thị hiển thị các Date tháng theo năm-tháng
years =YearLocator()
yearsFmt = DateFormatter('%Y')
months = MonthLocator()  # Thêm dòng này để khai báo MonthLocator
plt.gca().xaxis.set_major_locator(years)
plt.gca().xaxis.set_major_formatter(yearsFmt)
plt.gca().xaxis.set_minor_locator(months)

plt.tight_layout()
plt.show()

In [ ]:
df1 = pd.DataFrame(df,columns=['Date','Price'])
df1.index = df1.Date
df1.drop('Date',axis=1,inplace=True)
df1

In [ ]:
data = df1.values
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]
data

In [ ]:
#chuẩn hóa dữ liệu
sc = MinMaxScaler(feature_range=(0,1))
sc_train = sc.fit_transform(data)

#tạo vòng lặp các giá trị
x_train,y_train=[],[]
for i in range(60,len(train_data)):
  x_train.append(sc_train[i-60:i,0]) #lấy 50 giá đóng cửa liên tục
  y_train.append(sc_train[i,0]) #lấy ra giá đóng cửa ngày hôm sau

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
#xếp dữ liệu thành 1 mảng 2 chiều
x_train = np.array(x_train)
y_train = np.array(y_train)

#xếp lại dữ liệu thành mảng 1 chiều
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
y_train = np.reshape(y_train,(y_train.shape[0],1))

# Bước 5: Xây dựng và huấn luyện mô hình

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

model = Sequential()

# Lớp LSTM 1
model.add(LSTM(units=128, input_shape=(x_train.shape[1], 1), return_sequences=True))
model.add(BatchNormalization())  # Chuẩn hóa đầu ra
model.add(Dropout(0.3))  # Giảm dropout để tránh underfitting

# Lớp LSTM 2
model.add(LSTM(units=64))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Lớp đầu ra
model.add(Dense(1, activation='linear'))  # Rõ ràng chỉ định linear activation

# Tối ưu hóa với learning rate điều chỉnh
optimizer = Adam(learning_rate=0.0005)  # Giảm learning rate so với mặc định

# Biên dịch với cả MAE và MSE
model.compile(loss='mean_absolute_error',optimizer=optimizer, metrics=['mse'])  # Theo dõi thêm MSE

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss',patience=15, restore_best_weights=True)

model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint

# Huấn luyện lại model
save_model = "/drive/My Drive/DoAnDeepLearning/save_model.h5"
best_model = ModelCheckpoint(filepath=save_model,monitor='loss', verbose=2, save_best_only=True,mode='auto')

history = model.fit(x_train, y_train,epochs=500,batch_size=50,validation_split=0.2, verbose=2,callbacks=[best_model])
# Sau khi huấn luyện, lưu thủ công dưới dạng h5

model.save(save_model, save_format='h5')

In [ ]:
#dữ liệu train
y_train = sc.inverse_transform(y_train) #giá thực
final_model = load_model("/drive/My Drive/DoAnDeepLearning/save_model.h5")
y_train_predict = final_model.predict(x_train) #dự đoán giá đóng cửa trên tập đã train
y_train_predict = sc.inverse_transform(y_train_predict) #giá dự đoán

In [ ]:
import matplotlib.pyplot as plt

# Vẽ biểu đồ loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss', color='blue', marker='*', markevery=50)  # Thêm sao mỗi 50 epochs
plt.plot(history.history['val_loss'], label='Validation Loss', color='red', marker='o', markevery=50)  # Thêm dấu tròn
plt.title('Training History (Loss)', fontsize=14, pad=20)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

# Thêm chú thích điểm quan trọng
best_epoch = np.argmin(history.history['val_loss'])  # Epoch có val_loss thấp nhất
best_loss = np.min(history.history['val_loss'])
plt.scatter(best_epoch, best_loss, s=200, c='green', marker='*', label=f'Best Epoch ({best_epoch})')  # Đánh dấu sao to màu xanh

# Thêm text giải thích
plt.annotate(f'Val Loss Min: {best_loss:.4f}',
             xy=(best_epoch, best_loss),
             xytext=(best_epoch+50, best_loss+0.1),
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=12)

plt.legend()
plt.show()

# Bước 6: Tái sử dụng mô hình

In [ ]:
#xử lý dữ liệu test
test = df1[len(train_data)-60:].values
test = test.reshape(-1,1)
sc_test = sc.transform(test)

x_test = []
for i in range(60,test.shape[0]):
  x_test.append(sc_test[i-60:i,0])
x_test = np.array(x_test)
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

#dữ liệu test
y_test = data[train_size:] #giá thực
y_test_predict = final_model.predict(x_test)
y_test_predict = sc.inverse_transform(y_test_predict) #giá dự đoán

# Độ chính xác của mô hình

In [ ]:

# Lập biểu đồ so sánh
train_data1 = df1[60:train_size]  # Dữ liệu train sau khi đã xử lý cửa sổ trượt
test_data1 = df1[train_size:]

plt.figure(figsize=(24,8))
plt.plot(df1,label='Giá thực tế',color='red') #đường giá thực
train_data1['Dự đoán'] = y_train_predict #thêm dữ liệu
plt.plot(train_data1['Dự đoán'],label='Giá dự đoán train',color='green') #đường giá dự báo train
test_data1['Dự đoán'] = y_test_predict #thêm dữ liệu
plt.plot(test_data1['Dự đoán'],label='Giá dự đoán test',color='blue') #đường giá dự báo test
plt.title('So sánh giá dự báo và giá thực tế') #đặt tên biểu đồ
plt.xlabel('Thời gian') #đặt tên hàm x
plt.ylabel('Giá đóng cửa (USD)') #đặt tên hàm y
plt.legend() #chú thích
plt.show()

In [ ]:
print('Độ phù hợp tập train:', r2_score(y_train, y_train_predict))
print('Độ phù hợp tập test:', r2_score(y_test, y_test_predict))

In [ ]:
print('Sai số tuyệt đối trung bình trên tập train (USD):', mean_absolute_error(y_train, y_train_predict))
print('Sai số tuyệt đối trung bình trên tập test (USD):', mean_absolute_error(y_test, y_test_predict))

In [ ]:
print('Phần trăm sai số tuyệt đối trung bình tập train:', mean_absolute_percentage_error(y_train, y_train_predict))
print('Phần trăm sai số tuyệt đối trung bình tập test:', mean_absolute_percentage_error(y_test, y_test_predict))

In [ ]:
#r2
print('Độ phù hợp tập train:',r2_score(y_train,y_train_predict))
#mae
print('Sai số tuyệt đối trung bình trên tập train (USD):',mean_absolute_error(y_train,y_train_predict))
#mae
print('Phần trăm sai số tuyệt đối trung bình tập train:',mean_absolute_percentage_error(y_train,y_train_predict))

In [ ]:
train_data1

In [ ]:
#r2
print('Độ phù hợp tập test:',r2_score(y_test,y_test_predict))
#mae
print('Sai số tuyệt đối trung bình trên tập test (USD):',mean_absolute_error(y_test,y_test_predict))
#mae
print('Phần trăm sai số tuyệt đối trung bình tập test:',mean_absolute_percentage_error(y_test,y_test_predict))

In [ ]:
test_data1

# Thử dự đoán giá tương lai và so sánh với giá ngày cuối

In [ ]:
# Lấy ngày kế tiếp sau ngày cuối cùng trong tập dữ liệu để dự đoán
next_date = df['Date'].iloc[-1] + pd.Timedelta(days=1)

# Chuyển đổi ngày kế tiếp sang dạng datetime
next_date = pd.to_datetime(next_date)

# Lấy giá trị của ngày cuối cùng trong tập dữ liệu
next_closing_price = np.array([df['Price'].iloc[-1]])  # Lấy giá trị đóng cửa của ngày cuối cùng

# Chuẩn hóa giá trị của ngày cuối cùng
next_closing_price_normalized = sc.transform(next_closing_price.reshape(-1, 1))  # Chuyển thành mảng 2D

# Tạo dự đoán cho ngày kế tiếp bằng mô hình đã huấn luyện
x_next = np.array([sc_train[-50:, 0]])  # Lấy 50 giá đóng cửa gần nhất
x_next = np.reshape(x_next, (x_next.shape[0], x_next.shape[1], 1))
y_next_predict = final_model.predict(x_next)
y_next_predict = sc.inverse_transform(y_next_predict)

# Thêm dữ liệu dự đoán của ngày kế tiếp vào DataFrame
df_next = pd.DataFrame({'Ngày': [next_date], 'Đóng cửa': [y_next_predict[0][0]]})
df1 = pd.concat([df1, df_next])

# Vẽ biểu đồ mới với dự đoán cho ngày kế tiếp
plt.figure(figsize=(15, 5))
plt.plot(df1['Ngày'], df1['Đóng cửa'], label='Giá thực tế', color='red')
plt.plot(train_data1.index, train_data1['Dự đoán'], label='Giá dự đoán train', color='green')
plt.plot(test_data1.index, test_data1['Dự đoán'], label='Giá dự đoán test', color='blue')
plt.scatter([next_date], [y_next_predict[0][0]], color='orange', label='Dự đoán ngày kế tiếp')
plt.xlabel('Thời gian')
plt.ylabel('Giá đóng cửa (USD)')
plt.title('So sánh giá dự báo và giá thực tế')
plt.legend()
plt.show()


In [ ]:
# Lấy giá trị của ngày cuối cùng trong tập dữ liệu
actual_closing_price = df['Price'].iloc[-1]

# Tạo DataFrame so sánh giá dự đoán với giá ngày cuối trong tập dữ liệu
comparison_df = pd.DataFrame({'Ngày': [next_date], 'Giá dự đoán': [y_next_predict[0][0]], 'Giá ngày trước': [actual_closing_price]})

# In ra bảng so sánh
print(comparison_df)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(y_train, label='Actual', color='blue')
plt.plot(y_train_predict, label='Predicted', color='orange')
plt.title('Train Dataset')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()
